In [1]:
import os
import pickle
import uuid
from typing import Final, Optional

import cv2 as OpenCV
import numpy as np
from numpy.linalg import norm
from scipy.cluster.vq import kmeans, vq
from scipy.spatial import Delaunay

In [2]:
import sys

def print_size(obj):
    """
    Prints the memory usage of a Python object.

    Parameters:
    obj: Any Python object

    Returns:
    None
    """
    # Get the memory usage of the object in bytes
    memory_usage = sys.getsizeof(obj)

    # Convert memory usage to a more readable format
    if memory_usage < 1024:
        memory_usage_str = f"{memory_usage} bytes"
    elif memory_usage < 1024 ** 2:
        memory_usage_str = f"{memory_usage / 1024} KB"
    elif memory_usage < 1024 ** 3:
        memory_usage_str = f"{memory_usage / (1024 ** 2)} MB"
    else:
        memory_usage_str = f"{memory_usage / (1024 ** 3)} GB"

    # Print the memory usage
    print(f"Memory usage of the object: {memory_usage_str}")

def mem_usage():
    import psutil

    # Get the current process ID
    pid = psutil.Process()

    # Get the memory usage in bytes
    memory_usage = pid.memory_info().rss

    # Convert memory usage to GB
    memory_usage_gb = memory_usage / (1024 ** 3)

    # Print the memory usage in GB
    print(f"Memory usage: {memory_usage_gb:.2f} GB")

In [3]:
class CalibrationError(Exception):
    def __init__(self, message):
        self.message = message

class IntrinsicParametersNotFoundError(Exception):
    def __init__(self, message):
        self.message = message

In [4]:
class Image:
    def __init__(self, img_id, rgb_image, gray_image, keypoints, descriptors, path):
        self.img_id: int = img_id
        self.unique_id: uuid = uuid.uuid4()
        self.rgb_image: Image = rgb_image
        self.gray_image: Image = gray_image
        self.keypoints: list[OpenCV.KeyPoint] = keypoints
        self.descriptors: np.ndarray = descriptors
        self.path: str = path
        self.similar_images: list[tuple[Image, float]] = []

    @property
    def length(self):
        return f"{len(self.keypoints)}" if len(self.keypoints) == len(self.descriptors) else f"{len(self.keypoints)}, {len(self.descriptors)}"
    
    def __repr__(self):
        return f"Image({self.img_id})"
    
    def __str__(self):
        return self.__repr__()
    
    def __eq__(self, other):
        return self.unique_id == other.unique_id
    
    def __hash__(self):
        return hash(self.img_id)
    
    def __getstate__(self):
        state = self.__dict__.copy()
        state['keypoints'] = [tuple(k.pt) + (k.size, k.angle, k.response, k.octave, k.class_id) for k in self.keypoints]
        return state
    
    def __setstate__(self, state):
        state['keypoints'] = [OpenCV.KeyPoint(x, y, size, angle, response, octave, class_id) for x, y, size, angle, response, octave, class_id in state['keypoints']]
        self.__dict__ = state

class FeatureMatches:
    def __init__(self, image_one: Image, image_two: Image, matches: list[OpenCV.DMatch]):
        self.image_one: Image = image_one
        self.image_two: Image = image_two
        self.matches: list[OpenCV.DMatch] = matches

    def __repr__(self):
        return f"FeatureMatches({self.image_one}, {self.image_two} ---> {len(self.matches)})"

    def __getstate__(self):
        state = self.__dict__.copy()
        state['matches'] = [
            {'queryIdx': m.queryIdx, 'trainIdx': m.trainIdx, 'distance': m.distance} for m in self.matches
        ]
        return state
    
    def __setstate__(self, state):
        state['matches'] = [
            OpenCV.DMatch(match['queryIdx'], match['trainIdx'], match['distance']) for match in state['matches']
        ]
        self.__dict__ = state
    
class Images:
    def __init__(self, images: list[Image], image_set_name: str):
        self.id = uuid.uuid4()
        self.images: list[Image] = images
        self.image_set_name: str = image_set_name
        self.feature_matches: list[FeatureMatches] = []

    def __len__(self):
        return len(self.images)

In [5]:
""" Step One: Read and Load Images
Inputs: 
- folder_path: str

Outputs:
- images: Images

Main Functions:
1. prepare_images: read and load images from a folder into an Images object

Utils Functions:
1. dump_images: dump images to a pickle file
2. load_images: load images from a pickle file
"""

def prepare_images(folder_path: str) -> Images:
    """ Read and load images """
    images: Images = Images([], folder_path.split("/")[-1])
    files: list[str] = filter(lambda file: ".jpg" in file, os.listdir(folder_path))
    for i, file in enumerate(files):
        image_path = f"{folder_path}/{file}"
        rgb_image = OpenCV.cvtColor(OpenCV.imread(image_path), OpenCV.COLOR_BGR2RGB)
        gray_image = OpenCV.cvtColor(rgb_image, OpenCV.COLOR_RGB2GRAY)
        images.images.append(Image(i, rgb_image, gray_image, [], [], image_path))
    return images

def dump_images_bak(images_file_path: str, images: Images) -> None:
    """ Dump images to a file """
    with open(images_file_path, "wb") as file:
        pickle.dump(images, file)

def load_images_bak(images_file_path: str) -> Images:
    """ Load images from a file """
    with open(images_file_path, "rb") as file:
        images = pickle.load(file)
    return images

In [6]:
"""Step Two: Feature Extraction
Inputs:
- images: Images
- SIFT: OpenCV.SIFT

Outputs:
- image: Image
--> image.keypoints: list[OpenCV.KeyPoint]
--> image.descriptors: np.ndarray

Main Functions:
1. compute_keypoints_descriptors
"""

def compute_keypoints_descriptors(images: list[Image], SIFT: OpenCV.SIFT) -> None:
    """Compute keypoints and descriptors for each image in the list of images using SIFT algorithm.
    Modifies each image in the list of images by adding its keypoints and descriptors as attributes.
    
    Args:
    - images: List of images to compute keypoints and descriptors for.
    - SIFT: OpenCV SIFT object used to detect and compute keypoints and descriptors.

    Returns:
    - None.
    """
    for img in images.images:
        keypoints: list[OpenCV.KeyPoint]
        descriptors: np.ndarray
        keypoints, descriptors = SIFT.detectAndCompute(img.gray_image, None)
        img.keypoints = keypoints
        img.descriptors = descriptors

In [7]:
""" Step Three: Image Matching
Inputs:
- descriptors: list[np.ndarray]

Outputs:
- centroids: np.ndarray
- variance: np.ndarray
- CLUSTER_COUNT: int
- matches_ids: list[list[tuple[int, float]]]

Main Functions:
1. get_matches

Utils Functions:
1. get_matches_ids

Sub Utils Functions:
1. get_visual_words
2. get_frequency_vectors
3. get_tf_idf
4. search_matches
"""

def get_matches(images: Images) -> None:
    """ Match images using k-means clustering.
    Args:
        images: Obj from Images class.
    """
    all_descriptors = np.concatenate([image.descriptors for image in images.images])
    CLUSTER_COUNT: Final = 400
    ITER: Final = 2
    centroids, _ = kmeans(all_descriptors, CLUSTER_COUNT, ITER)
    matches_ids: list[list[tuple[int, float]]] =  get_matches_ids([image.descriptors for image in images.images], centroids, images.images)
    for i, image in enumerate(images.images):
        inner_list: list[Image, float] = [
            (images.images[match[0]], match[1]) for match in matches_ids[i]
        ]
        image.similar_images = inner_list


def get_visual_words(descriptors: list[np.ndarray], centroids: np.ndarray) -> list[np.ndarray]:
    """ Get the visual words of a list of descriptors.
    Args:
        descriptors: A list of numpy arrays containing image descriptors.
        centroids: A numpy array containing cluster centroids.
    Returns:
        A list of numpy arrays representing the visual words of each image.
    """
    visual_words = []
    for descriptor in descriptors:
        words, _ = vq(descriptor, centroids)
        visual_words.append(words)
    return visual_words


def get_frequency_vectors(visual_words: list[np.ndarray], CLUSTER_COUNT: int) -> np.ndarray:
    """ Get the frequency vectors for a list of visual words.
    Args:
        visual_words: A list of numpy arrays representing the visual words of each image.
        CLUSTER_COUNT: The number of clusters used to generate the visual words.
    Returns:
        A numpy array containing the frequency vectors for each image.
    """
    frequency_vectors = []
    for img_words in visual_words:
        histogram = np.zeros(CLUSTER_COUNT)
        for word in img_words:
            histogram[word] += 1
        frequency_vectors.append(histogram)
    return np.stack(frequency_vectors)


def get_tf_idf(frequency_vectors, IMAGES_COUNT) -> np.ndarray:
    """ Get the Term Frequency-Inverse Document Frequency (TF-IDF) matrix for a list of frequency vectors.
    Args:
        frequency_vectors: A numpy array containing the frequency vectors for each image.
        IMAGES_COUNT: The total number of images in the dataset.
    Returns:
        A numpy array containing the TF-IDF matrix for the input frequency vectors.
    """
    df = np.sum(frequency_vectors > 0, axis = 0)
    idf = np.log(IMAGES_COUNT/df)
    return frequency_vectors * idf


def search_matches(i, top_clusters, tf_idf) -> list[tuple[int, float]]:
    """ Search for the top_clusters most similar images to the i-th image
    Args:
        i: the index of the image to search for similar images
        top_clusters: the number of similar images to return
        tf_idf: Term Frequency-Inverse Document Frequency
    Returns:
        A list of tuples, where each tuple contains the index of a similar image and the cosine similarity 
        between the i-th image and the similar image. The list is sorted by the cosine similarity in 
        descending order.
    """
    b = tf_idf
    a = tf_idf[i]
    b_subset = b[:tf_idf.shape[0]]
    cosine_similarity = np.dot(a, b_subset.T)/(norm(a) * norm(b_subset, axis=1))
    idx = np.argsort(-cosine_similarity)[:top_clusters]
    return list(zip(idx, cosine_similarity[idx]))


def get_matches_ids(descriptors, centroids, images_list) -> list[list[tuple[int, float]]]:
    """Returns: a list of lists, where each list contains the top 10 most similar images to the i-th image."""
    visual_words = get_visual_words(descriptors, centroids)
    frequency_vectors = get_frequency_vectors(visual_words, centroids.shape[0])
    """ tf_idf: Term Frequency-Inverse Document Frequency """
    tf_idf = get_tf_idf(frequency_vectors, len(images_list))
    return [
        search_matches(i, 10, tf_idf)
        for i in range(len(images_list))
    ]

In [8]:
"""Step Four: Feature Matching
Inputs:
- images: Images

Outputs:
- None

Main Functions:
1. data_feature_matching

Utils Functions:
1. feature_matching
"""

def feature_matching(
        img_one_descriptors: np.ndarray, 
        img_two_descriptors: np.ndarray
    ) -> list[OpenCV.DMatch]:
    """ Match features between two images using Brute Force Matcher
    Args:
        img_id_one: the index of the first image
        img_id_two: the index of the second image
        descriptors: a list of descriptors of the images
    Returns:
        A list of OpenCV.DMatch objects.
    """
    matcher = OpenCV.BFMatcher()
    return matcher.match(img_one_descriptors, img_two_descriptors)

def data_feature_matching(images: Images) -> None:
    """ Match features between images using Brute Force Matcher
    Args:
        matchesIDs: a list of lists of tuples, where each tuple contains the index of a similar image and the cosine similarity 
            between the i-th image and the similar image. The list is sorted by the cosine similarity in 
            descending order.
        descriptors: a list of descriptors of the images
    Returns:
        A list of lists, where each list contains 
        the index of the first image, the index of the second image, 
        and a list of OpenCV.DMatch objects.
    """
    num_images: int = len(images.images)
    checked = np.zeros((num_images, num_images), dtype=int)
    for image in images.images:
        for matched_image, probability in image.similar_images:
            if ((checked[image.img_id][matched_image.img_id] == 0 or checked[matched_image.img_id][image.img_id] == 0) and image.img_id != matched_image.img_id and probability > 0.93):
                images.feature_matches.append(FeatureMatches(image, matched_image, feature_matching(image.descriptors, matched_image.descriptors)))
                checked[image.img_id][matched_image.img_id], checked[matched_image.img_id][image.img_id] = 1, 1

In [9]:
"""Step Five: Camera Calibration
Inputs:
- None.

Outputs:
- k_matrix: np.ndarray

ToDo:
1- generate K_matrix.pickle for each camera using Chess board pattern.
"""

'Step Five: Camera Calibration\nInputs:\n- None.\n\nOutputs:\n- k_matrix: np.ndarray\n\nToDo:\n1- generate K_matrix.pickle for each camera using Chess board pattern.\n'

In [10]:
"""Step Six: Triangulation (3D Reconstruction)
Inputs:
- feature_matches_list: list[list[int, int, list[OpenCV.DMatch]]]
    -> A list of lists, where each list contains 
        the index of the first image, the index of the second image, 
        and a list of OpenCV.DMatch objects.
- K_matrix: np.ndarray
    -> The camera matrix of the camera used to take the images.

Outputs:
- point_cloud: list[np.ndarray]; each element is a 3D point.

Main Functions:
1. generate_point_cloud

Utils Functions:
1. triangulatePoints
"""

def triangulatePoints(P1, P2, pts1, pts2):
    """
    Triangulates the given matching points from two images using the given camera matrices.

    Parameters:
    P1 (numpy.ndarray): 3x4 camera matrix of the first image.
    P2 (numpy.ndarray): 3x4 camera matrix of the second image.
    pts1 (numpy.ndarray): Nx2 matrix containing the coordinates of matching points in the first image.
    pts2 (numpy.ndarray): Nx2 matrix containing the coordinates of matching points in the second image.

    Returns:
    numpy.ndarray: Nx3 matrix containing the triangulated 3D points.
    """
    pts4D = OpenCV.triangulatePoints(P1, P2, pts1.T, pts2.T)
    pts4D /= pts4D[3]
    return pts4D[:3].T


def generate_point_cloud(images: Images, K_matrix):
    """
    Generates a cloud of 3D points using triangulation from feature matches and camera calibration matrix.

    Parameters:
    feature_matches_list (list): List of feature matches between images.
    K_matrix (numpy.ndarray): 3x3 camera calibration matrix.

    Returns:
    numpy.ndarray: Nx3 matrix containing the cloud of 3D points.
    """
    point_cloud = []
    feature_matches_list = images.feature_matches
    for match in feature_matches_list:
        img_one = match.image_one
        img_two = match.image_two
        matches = match.matches
        pts1 = np.float32([img_one.keypoints[m.queryIdx].pt for m in matches])
        pts2 = np.float32([img_two.keypoints[m.trainIdx].pt for m in matches])
        E, _ = OpenCV.findEssentialMat(pts1, pts2, K_matrix)
        R1, R2, t = OpenCV.decomposeEssentialMat(E)
        P1 = np.hstack((np.eye(3), np.zeros((3,1))))
        P2 = np.hstack((R1, t))
        pts_3d = triangulatePoints(P1, P2, pts1, pts2)
        point_cloud.append(pts_3d)
    return np.concatenate(point_cloud, axis=0)


In [11]:
print("Welcome ScanMate...")
# image_set_name = "rubik-cube"
image_set_name = "snow-man"
images: Optional[Images] = None
# Reload the last state
last_state: str
if os.path.isfile(f"bak/{image_set_name}/point-cloud.pkl"):
    last_state = "Point Cloud Step"
elif os.path.isfile(f"bak/{image_set_name}/feature-matching-output.pkl"):
    last_state = "Feature Matching Step"
elif os.path.isfile(f"bak/{image_set_name}/images-matched.pkl"):
    last_state = "Images Matching Step"
elif os.path.isfile(f"bak/{image_set_name}/sift-features.pkl"):
    last_state = "SIFT Features Step"
else:
    last_state = "Images Loading Step"

Welcome ScanMate...


In [12]:
print(last_state)

Point Cloud Step


In [13]:
last_state = "Images Loading Step"

In [14]:
# 1. Load and prepare Images
if last_state == "Images Loading Step":
    if os.path.isfile(f"bak/{image_set_name}/images.pkl"):
        print(f"File [bak/{image_set_name}/sift-images.pkl] exists")
        print("Loading images from pickle file...")
        images: Images = load_images_bak(f"bak/{image_set_name}/images.pkl")
    else:
        print(f"File [bak/{image_set_name}/images.pkl] does not exist")
        print("Loading images from images directory...")
        images: Images = prepare_images(f"images/{image_set_name}")
        print("Saving images to pickle file...")
        dump_images_bak(f"bak/{image_set_name}/images.pkl", images)
    print("Images loaded successfully")
    last_state = "SIFT Features Step"

File [bak/snow-man/sift-images.pkl] exists
Loading images from pickle file...
Images loaded successfully


In [15]:
from collections import Counter

print(Counter(len(image.descriptors) for image in images.images))

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [16]:
# 2. Feature Extraction: SIFT
if last_state == "SIFT Features Step":
    if os.path.isfile(f"bak/{image_set_name}/sift-features.pkl"):
        print(f"File [bak/{image_set_name}/sift-features.pkl] exists")
        if images: 
            del images
        images: Images = load_images_bak(f"bak/{image_set_name}/sift-features.pkl")
    else:
        print("File [bak/{image_set_name}/sift-features.pkl] DO NOT exists")
        print("Extracting SIFT features...")
        sift = OpenCV.SIFT_create()
        compute_keypoints_descriptors(images, sift)
        dump_images_bak(f"bak/{image_set_name}/sift-features.pkl", images)
        # remove bak/{image_set_name}/images.pkl
        if os.path.exists(f"bak/{image_set_name}/images.pkl"):
            os.remove(f"bak/{image_set_name}/images.pkl")
            print(f"File bak/{image_set_name}/images.pkl removed successfully.")
        else:
            print(f"File bak/{image_set_name}/images.pkl does not exist.")
    print("Feature Extraction: SIFT DONE...")
    last_state = "Images Matching Step"

File [bak/{image_set_name}/sift-features.pkl] DO NOT exists
Extracting SIFT features...
File bak/snow-man/images.pkl removed successfully.
Feature Extraction: SIFT DONE...


In [18]:
from collections import Counter

print(Counter(len(image.descriptors) for image in images.images))

Counter({30209: 1, 23071: 1, 12823: 1, 7230: 1, 22413: 1, 3357: 1, 8848: 1, 3925: 1, 42699: 1, 24060: 1, 12652: 1, 4665: 1, 1066: 1, 4034: 1, 1411: 1, 8548: 1, 6014: 1, 1464: 1, 9389: 1, 20041: 1, 19830: 1, 6859: 1, 3894: 1, 12667: 1, 6466: 1, 3149: 1, 5084: 1, 52593: 1, 102355: 1, 32483: 1, 23916: 1, 5067: 1, 42670: 1, 30971: 1, 26247: 1, 37630: 1, 42297: 1, 50236: 1, 16449: 1, 12537: 1, 6261: 1, 4626: 1, 1141: 1, 1508: 1, 11611: 1, 72755: 1, 137129: 1, 110434: 1, 18104: 1, 60194: 1, 28468: 1, 7706: 1, 7005: 1, 12665: 1, 8682: 1, 8702: 1, 33066: 1, 25576: 1, 75419: 1, 19479: 1, 29750: 1, 34953: 1, 4640: 1, 57841: 1, 109502: 1, 29821: 1, 4255: 1, 7733: 1, 2413: 1, 12692: 1, 23020: 1, 77360: 1, 97377: 1, 88101: 1, 35749: 1, 9190: 1, 49983: 1, 27149: 1, 13494: 1, 5050: 1, 15210: 1, 5178: 1, 5287: 1, 3936: 1, 15374: 1, 25733: 1, 2551: 1, 4290: 1, 6748: 1, 4524: 1, 3766: 1, 12856: 1, 5138: 1, 4136: 1, 2854: 1, 5692: 1, 4525: 1, 8908: 1, 10857: 1, 15784: 1, 20075: 1, 9188: 1, 8359: 1, 7740:

In [17]:
# 3. Image Matching
if last_state == "Images Matching Step":
    if os.path.isfile(f"bak/{image_set_name}/images-matched.pkl"):
        print(f"File [bak/{image_set_name}/images-matched.pkl] exists")
        if images: 
            del images
        images: Images = load_images_bak(f"bak/{image_set_name}/images-matched.pkl")
    else:
        print(f"File [bak/{image_set_name}/images-matched.pkl] DO NOT exists")
        get_matches(images)
        dump_images_bak(f"bak/{image_set_name}/images-matched.pkl", images)
        # remove bak/{image_set_name}/sift-features.pkl
        if os.path.exists(f"bak/{image_set_name}/sift-features.pkl"):
            os.remove(f"bak/{image_set_name}/sift-features.pkl")
            print(f"File bak/{image_set_name}/sift-features.pkl removed successfully.")
        else:
            print(f"File bak/{image_set_name}/sift-features.pkl does not exist.")
    print("Done Image Matching Step...")
    last_state = "Feature Matching Step"

File [bak/snow-man/images-matched.pkl] DO NOT exists


KeyboardInterrupt: 

In [ ]:
# 4. Feature Matching
if last_state == "Feature Matching Step":
    if os.path.isfile(f"bak/{image_set_name}/feature-matching-output.pkl"):
        print(f"File [bak/{image_set_name}/feature-matching-output.pkl] exists")
        if images: 
            del images
        images: Images = load_images_bak(f"bak/{image_set_name}/feature-matching-output.pkl")
    else:
        print("File [bak/{image_set_name}/feature-matching-output.pkl] Do NOT exists")
        # logging.info('----> Processing {image_set_name}...')
        data_feature_matching(images)
        dump_images_bak(f"bak/{image_set_name}/feature-matching-output.pkl", images)
        # remove bak/{image_set_name}/images-matched.pkl
        if os.path.exists(f"bak/{image_set_name}/images-matched.pkl"):
            os.remove(f"bak/{image_set_name}/images-matched.pkl")
            print(f"File bak/{image_set_name}/images-matched.pkl removed successfully.")
        else:
            print(f"File bak/{image_set_name}/images-matched.pkl does not exist.")
    print("Done Feature Matching Step...")
    last_state = "Point Cloud Step"

In [ ]:
# 5. Camera Calibration
print("Camera Calibration starts ....")
if not os.path.isfile("bak/snow-man/checker/K_matrix.pickle"):
    raise IntrinsicParametersNotFoundError("Intrinsic parameters not found")
print("File bak/snow-man/checker/K_matrix.pickle exists")
with open('bak/snow-man/checker/K_matrix.pickle', 'rb') as f:
    K_matrix = pickle.load(f)

In [ ]:
# 6. Triangulation (3D reconstruction)
print("Triangulation starts ....")
if last_state == "Point Cloud Step":
    if os.path.isfile(f"bak/{image_set_name}/point-cloud.pkl"):
        with open(f"bak/{image_set_name}/point-cloud.pkl", 'rb') as f:
            points_cloud: np.ndarray = pickle.load(f)
    else:
        points_cloud: np.ndarray = generate_point_cloud(images, K_matrix)
        # Pickle the point cloud
        with open(f"bak/{image_set_name}/point-cloud.pkl", 'wb') as f:
            pickle.dump(points_cloud, f)
        # remove bak/{image_set_name}/feature-matching-output.pkl
        # if os.path.exists(f"bak/{image_set_name}/feature-matching-output.pkl"):
        #     os.remove(f"bak/{image_set_name}/feature-matching-output.pkl")
        #     print(f"File bak/{image_set_name}/feature-matching-output.pkl removed successfully.")
        # else:
        #     print(f"File bak/{image_set_name}/feature-matching-output.pkl does not exist.")

In [ ]:
# # Free Memory from images (not tested yet)
# mem_usage()

# if images: 
#   print_size(images)
#   del images
#   print("We freed the memory")
# else:
#   print("Memory was not freed")

# print_size(points_cloud)

# mem_usage()

In [ ]:
print(*(len(image.descriptors) for image in images.images))

In [ ]:
print(f"points_cloud.shape: {points_cloud.shape}")
# Save the point cloud to a file
# np.savetxt(f"output/{image_set_name}/point-cloud.txt", points_cloud)

# Start of Debugging

In [ ]:
# # plot points_cloud
# import matplotlib.pyplot as plt
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(points_cloud[:,0], points_cloud[:,1], points_cloud[:,2])

# ax.set_xlabel('X Label')
# ax.set_ylabel('Y Label')
# ax.set_zlabel('Z Label')

# plt.show()
# plt.savefig('points_histo_plt.png')

### Statics about data

In [ ]:
from collections import Counter

# Log to tune.log
def log_tune(file_name: str, message: str):
    with open(file_name, "a") as f:
        f.write(f"{message}\n")

log_tune("tune-before.log", f"X<{len(points_cloud[:,0]):,}>: {points_cloud[:,0].min():,} to {points_cloud[:,0].max():,}")
x_counter = Counter(points_cloud[:,0])
log_tune("tune-before.log", f"We have {len(x_counter):,} unique Y values")
log_tune("tune-before.log", f"Most Common X: {x_counter.most_common(1)}, Least Two Common X: {x_counter.most_common()[:-3:-1]}")
# log_tune("tune-before.log", x_counter)
log_tune("tune-before.log", "-----------------------------------------------------")
log_tune("tune-before.log", f"Y<{len(points_cloud[:,1]):,}>: {points_cloud[:,1].min():,} to {points_cloud[:,1].max():,}")
y_counter = Counter(points_cloud[:,1])
log_tune("tune-before.log", f"We have {len(y_counter):,} unique Y values")
log_tune("tune-before.log", f"Most Common Y: {y_counter.most_common(1)}, Least Two Common Y: {y_counter.most_common()[:-3:-1]}")
# log_tune("tune-before.log", y_counter)
log_tune("tune-before.log", "-----------------------------------------------------")
log_tune("tune-before.log", f"Z<{len(points_cloud[:,2]):,}>: {points_cloud[:,2].min():,} to {points_cloud[:,2].max():,}")
z_counter = Counter(points_cloud[:,2])
log_tune("tune-before.log", f"We have {len(z_counter):,} unique Z values")
log_tune("tune-before.log", f"Most Common Z: {z_counter.most_common(1)}, Least Two Common Y: {z_counter.most_common()[:-3:-1]}")

In [ ]:
# new_points_cloud = [
#     point
#     for point in points_cloud
#     if -10 <= point[0] <= 10
#     and -10 <= point[1] <= 10
#     and -10 <= point[2] <= 10
# ]

# log_tune("tune-after.log", f"Now we have {len(new_points_cloud):,} points")
# new_points_cloud = np.array(new_points_cloud)

# # scale z-axis by factor of 10_000
# new_points_cloud[:, 2] = new_points_cloud[:, 2] * 10_000

# log_tune("tune-after.log", f"X<{len(new_points_cloud[:,0]):,}>: {new_points_cloud[:,0].min():,} to {new_points_cloud[:,0].max():,}")
# new_x_counter = Counter(new_points_cloud[:,0])
# log_tune("tune-after.log", f"We have {len(new_x_counter):,} unique Y values")
# log_tune("tune-after.log", f"Most Common X: {new_x_counter.most_common(1)}, Least Two Common X: {new_x_counter.most_common()[:-3:-1]}")
# # log_tune("tune-after.log", x_counter)
# log_tune("tune-after.log", "-----------------------------------------------------")
# log_tune("tune-after.log", f"Y<{len(new_points_cloud[:,1]):,}>: {new_points_cloud[:,1].min():,} to {new_points_cloud[:,1].max():,}")
# new_y_counter = Counter(new_points_cloud[:,1])
# log_tune("tune-after.log", f"We have {len(new_y_counter):,} unique Y values")
# log_tune("tune-after.log", f"Most Common Y: {new_y_counter.most_common(1)}, Least Two Common Y: {new_y_counter.most_common()[:-3:-1]}")
# # log_tune("tune-after.log", y_counter)
# log_tune("tune-after.log", "-----------------------------------------------------")
# log_tune("tune-after.log", f"Z<{len(new_points_cloud[:,2]):,}>: {new_points_cloud[:,2].min():,} to {new_points_cloud[:,2].max():,}")
# new_z_counter = Counter(new_points_cloud[:,2])
# log_tune("tune-after.log", f"We have {len(new_z_counter):,} unique Z values")
# log_tune("tune-after.log", f"Most Common Z: {new_z_counter.most_common(1)}, Least Two Common Y: {new_z_counter.most_common()[:-3:-1]}")

In [ ]:
mem_usage()

print_size(points_cloud)
print_size(x_counter)
print_size(y_counter)
print_size(z_counter)

# print_size(new_points_cloud)
# print_size(new_x_counter)
# print_size(new_y_counter)
# print_size(new_z_counter)

# del points_cloud
del x_counter
del y_counter
del z_counter

# del new_points_cloud
# del new_x_counter
# del new_y_counter
# del new_z_counter

mem_usage()

In [ ]:
# from sklearn.cluster import DBSCAN
# # Cluster the points using DBSCAN algorithm
# dbscan = DBSCAN(eps=0.5, min_samples=10).fit(points_cloud)

# # Get the cluster labels for each point
# labels = dbscan.labels_

# # Print all labels
# print("All labels:", labels)

# # Get the indices of the core points (i.e., points that are part of a dense region)
# core_indices = np.where(labels != -1)[0]

# # Get the coordinates of the core points
# core_points = points_cloud[core_indices, :]

# # Get the indices of the outlier points (i.e., points that are not part of any dense region)
# outlier_indices = np.where(labels == -1)[0]

# # Get the coordinates of the outlier points
# outlier_points = points_cloud[outlier_indices, :]

# # Print the number of clusters and the number of outlier points
# print("Number of clusters:", len(np.unique(labels)) - 1)
# print("Number of outlier points:", len(outlier_indices))

### Plotting the histogram of the data


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# NUM_PINS: Final[int] = 2_000_000

In [ ]:
# # x_counter = {k: v for k, v in x_counter.items() if -4 < k < 4}
# # new_x_counter = Counter(x_counter)
# print(f"We have {len(new_x_counter):,} unique X values")
# print(f"Most Common X: {new_x_counter.most_common(1)}, Least Two Common X: {new_x_counter.most_common()[:-3:-1]}")
# print(f"Max X: {max(new_x_counter)}, Min X: {min(new_x_counter)}")
# plt.hist(list(new_x_counter.keys()), weights=list(new_x_counter.values()), bins=NUM_PINS, label='x-axis')
# plt.title('X Points Cloud Histogram')
# plt.xlabel('Point X Value')
# plt.ylabel('Frequency')
# plt.savefig('x_points_histo.png')

In [ ]:
# # # plot points_cloud
# # import matplotlib.pyplot as plt
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(new_points_cloud[:,0], new_points_cloud[:,1], new_points_cloud[:,2])

# ax.set_xlabel('X Label')
# ax.set_ylabel('Y Label')
# ax.set_zlabel('Z Label')

# plt.show()
# plt.savefig('points_histo_plt.png')

In [ ]:
# # y_counter = {k: v for k, v in y_counter.items() if -10 < k < 10}
# # y_counter = Counter(y_counter)
# print(f"We have {len(new_y_counter):,} unique Y values")
# print(f"Most Common Y: {new_y_counter.most_common(1)}, Least Two Common Y: {new_y_counter.most_common()[:-3:-1]}")
# print(f"Max Y: {max(new_y_counter)}, Min Y: {min(new_y_counter)}")
# plt.hist(list(new_y_counter.keys()), weights=list(new_y_counter.values()), bins=NUM_PINS, label='y-axis')
# plt.title('Y Points Cloud Histogram')
# plt.xlabel('Point Y Value')
# plt.ylabel('Frequency')
# plt.savefig('y_points_histo.png')

In [ ]:
# mem_usage()

# new_z_counter = {k: v for k, v in new_z_counter.items() if -0.001 < k < 0.001}
# new_z_counter = Counter(new_z_counter)
# print(f"We have {len(new_z_counter):,} unique Z values")
# print(f"Most Common Z: {new_z_counter.most_common(1)}, Least Two Common Z: {new_z_counter.most_common()[:-3:-1]}")
# print(f"Max Z: {max(new_z_counter)}, Min Z: {min(new_z_counter)}")
# plt.hist(list(new_z_counter.keys()), weights=list(new_z_counter.values()), bins=NUM_PINS, label='z-axis')
# plt.title('Z Points Cloud Histogram')
# plt.xlabel('Point Z Value')
# plt.ylabel('Frequency')
# plt.savefig('z_points_histo.png')

# mem_usage()
# del plt
# mem_usage()

In [ ]:
# def log_to_file(filename: str, message: str):
#     with open(filename, "a") as f:
#         f.write(f"{message}\n")

# def print_ascii_bar_chart(data, filename, symbol="#"):
#     counter = Counter(data).most_common()
#     chart = {category: symbol * frequency for category, frequency in counter}
#     max_len = max(len(category) for category in chart)
#     for category, frequency in chart.items():
#         padding = (max_len - len(category)) * " "
#         log_to_file(filename, f"{category}{padding} |{frequency}")
# print_ascii_bar_chart(points_cloud[:,0], "x_values_freq.log", symbol="+")
# print_ascii_bar_chart(points_cloud[:,1], "y_values_freq.log", symbol="+")
# print_ascii_bar_chart(points_cloud[:,2], "z_values_freq.log", symbol="+")

In [ ]:
# import matplotlib.pyplot as plt
# fig, axs = plt.subplots(1, 3, sharey=True, tight_layout=True)
# axs[0].hist(points_cloud[:, 0], bins=17_000_000)
# axs[1].hist(points_cloud[:, 1], bins=17_000_000)
# axs[2].hist(points_cloud[:, 2], bins=17_000_000)
# plt.show()

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# # Generate point cloud
# # point_cloud = np.random.rand(10000, 3)

# # Define scaling factor
# scale_factor = 2.0

# # Scale the point cloud
# # scaled_point_cloud = scaled_point_cloud * scale_factor

# # Plot histogram of scaled_point_cloud
# plt.hist(points_cloud.flatten(), bins=17_000_000, alpha=0.5, label='Before Scaling')
# # Plot histogram of scaled_point_cloud
# # plt.hist(scaled_point_cloud.flatten(), bins=50, alpha=0.5, label='After Scaling')

# # Add titles and labels to the plot
# plt.title('Point Cloud Histogram')
# plt.xlabel('Point Value')
# plt.ylabel('Frequency')
# plt.legend(loc='upper right')

# # Show the plot
# plt.show()

In [ ]:
# temp_points_cloud = new_points_cloud

# new_points_cloud = [
#     point
#     for point in temp_points_cloud
#     if -10 <= point[0] <= 10
#     and -10 <= point[1] <= 10
#     and -10 <= point[2] <= 10
# ]

# log_tune("tune-after.log", f"Now we have {len(new_points_cloud):,} points")
# new_points_cloud = np.array(new_points_cloud)

# END of Debugging

In [ ]:
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_cloud[:,:3])

In [ ]:
# Save it as a.STL file
o3d.io.write_point_cloud("point_cloud.ply", pcd)

In [ ]:
# o3d.visualization.draw_geometries([pcd])